In [ ]:
import pymongo
from pymongo import MongoClient
from bson import SON

from IPython.html.widgets import FloatProgress
from IPython.display import display

In [ ]:
client = MongoClient()
db = client['gplaces']
placesColl = db['places']
places_geojson = db['places_geojson']

In [168]:
#types = 'name,rating,types,user_ratings_total'
for locTemp in placesColl.find():
    newField = {'loc':{ 'type': "Point",'coordinates': [locTemp['geometry']['location']['lat'], locTemp['geometry']['location']['lng']]}}
    
    if 'name' in locTemp:
        newField['name'] = locTemp['name']
    
    if 'rating' in locTemp:
        newField['rating'] = locTemp['rating']
        
    if 'user_ratings_total' in locTemp:
        newField['user_ratings_total'] = locTemp['user_ratings_total']
        
    if 'types' in locTemp:
        newField['types'] = locTemp['types']

    places_geojson.insert_one(newField)

In [182]:
places_geojson.find_one()

{'_id': ObjectId('571277bf6fe8b929a346aa56'),
 'loc': {'coordinates': [59.83275469999999, 30.2317329], 'type': 'Point'},
 'name': 'Александрино',
 'rating': 4.4,
 'types': ['park', 'point_of_interest', 'establishment'],
 'user_ratings_total': 47}

In [201]:
#DUMP

import json
from bson.json_util import dumps

json_data = list()

for item in places_geojson.find().limit(30000):
    item_dict = dict()
    item_dict['lng'] = item['loc']['coordinates'][0]
    item_dict['lat'] = item['loc']['coordinates'][1]
    item_dict['count'] = 1
    json_data.extend([item_dict])


with open('data_100_2.json', 'w') as fp:
    json.dump(json_data, fp)
    
#with open('data_100.json', 'w') as outfile:
#    json.dump(dumps(places_geojson.find().limit(100)), outfile)

In [ ]:
places_geojson.remove({})

In [ ]:
places_geojson.create_index([('loc' , pymongo.GEOSPHERE)])

In [200]:
result = places_geojson.aggregate([
   {
     '$geoNear': {
        'near': { 'type': "Point", 'coordinates': [59.83275469999999, 30.2317329] },
        'distanceField': "dist",
        'maxDistance': 550,
        'spherical': True
     }
   }
])


for i,item in enumerate(result):
    print(item)
    if i > 20:
        break

{'_id': ObjectId('571277bf6fe8b929a346aa56'), 'user_ratings_total': 47, 'loc': {'type': 'Point', 'coordinates': [59.83275469999999, 30.2317329]}, 'types': ['park', 'point_of_interest', 'establishment'], 'name': 'Александрино', 'dist': 0.0, 'rating': 4.4}
{'loc': {'type': 'Point', 'coordinates': [59.837781, 30.23042659999999]}, 'dist': 504.8250549746493, 'name': 'Zaprav.com, servis-tsentr', '_id': ObjectId('571277bf6fe8b929a346aa89'), 'types': ['point_of_interest', 'establishment']}
{'_id': ObjectId('571277c16fe8b929a346b1bc'), 'user_ratings_total': 1, 'loc': {'type': 'Point', 'coordinates': [59.837781, 30.23042659999999]}, 'name': 'Hair`isma, salon krasoty', 'dist': 504.8250549746493, 'types': ['point_of_interest', 'establishment']}
{'loc': {'type': 'Point', 'coordinates': [59.8380414, 30.23114229999999]}, 'dist': 512.7040482724618, 'name': 'Zolotaya Rybka', '_id': ObjectId('571277bf6fe8b929a346aa59'), 'types': ['pet_store', 'store', 'point_of_interest', 'establishment']}
{'loc': {'typ